# Trax Interactive Tutorial

This tutorial was created by [Alex Bagi](https://github.com/alba1735)

---
## About tRAX

tRNA Analysis of eXpression (tRAX) is a software package built for in-depth analyses of tRNA-derived small RNAs (tDRs), mature tRNAs, and inference of RNA modifications from high-throughput small RNA sequencing data. While the tRAX workflow adopts popular RNA sequencing data analysis methods, which includes adapter trimming of raw sequencing data, read alignment to the reference, transcript abundance estimation, and differential expression analysis among samples, it specifically consists of features designed to support special characteristics of tRNAs and tDRs. To ensure alignment of sequencing reads to tRNA transcripts, tRAX uses a custom-built reference database that not only includes the reference genome but also mature tRNA transcripts with the addition of 3′ CCA tail not encoded genomically. Unlike popular read counting tools that only consider or recommend for uniquely mapped reads for RNA sequencing, tRAX allows reads to be mapped to multiple transcripts and gene loci, which is necessary for conserved tRNA isodecoders (tRNAs with the same anticodon but different sequences in the gene body) and identical tRNA genes that are commonly found in eukayotes. Read coverage for each tRNA transcript is reported in four categories – transcript-specific, isodecoder-specific, isotype-specific, and non-specific – that provide precise results on the level of uniqueness of the aligned reads. Moreover, tRAX computes separate read counts of tRNA fragments that align at the 5′ end, 3′ end, and the middle region of tRNA transcripts to distinguish the abundance of different fragment types. Differential expression comparison across samples is performed using read counts for tRNA transcripts and tRNA fragments to provide better understanding of possible distinction in different tRNA isotypes or isodecoders and fragment types. In addition, tRAX measures the base frequency at each position aligned to the tRNA transcripts for estimating the mis-incorporations that may represent RNA modifications essential for function, stability, and regulation.

[http://trna.ucsc.edu/tRAX/](http://trna.ucsc.edu/tRAX/)

## About Google Colab
Google Colab is a free notebook-based environment that can be used to run the tRAX pipeline for small datasets. Since the free version of Colab is limited to 2 CPU cores and 12 GB of ram, you can also connect it to a [local runtime](https://research.google.com/colaboratory/local-runtimes.html) (your local machine) however, we have not tested this. This notebook can also be modified and run via [Jupyter Notebook/Lab](https://jupyter.org/) on a local machine or server.

# Setting up Colab w/ conda for TRAX

This Colab notebook is designed to be an easy-to-use interactive tutorial for tRAX. All the commands in this notebook can be applied as guidelines on how to run tRAX. One thing to note is that any "`!`" found before lines of code is a Colab-specific notation to tell the cell to run that code with the shell rather than with Python3 (On a Linux server, you would omit these).

* Add a shared folder to your drive by selecting the "Add to My Drive" option under "Shared With Me" or launch this notebook in Playground mode
  * Alternatively if coming from GitHub you can just upload this notebook to [Colab](https://colab.research.google.com/) and run it that way
* [Conda](https://docs.conda.io/projects/conda/en/latest/index.html) is a convenient way to set up the many dependencies required to run TRAX. A modified version of conda is setup in this collab notebook with the following:
  ```
  !pip install -q condacolab
  import condacolab
  condacolab.install() # expect a kernel restart
  ```
  * **Running this first cell will cause the kernel to restart this is normal!**
  * **DO NOT** run all cells at the top, or you will get errors. Instead, run this cell, and then after the kernel restarts, you can proceed to run all subsequent cells with "run after" under "runtime" at the top.
* The next cell will mount Google Grive on Colab and open a pop-up window. Click on the link from the above prompt and choose your Google account for storage access:
  ```
  from google.colab import drive
  drive.mount('/content/drive')
  ```
* Datasets that you can use with Python's scanpy are located under `/content/drive/MyDrive/<YOUR GOOGLE DRIVE>`
* A folder will be created in your Google Drive called `trax_tutorial` that will be used to store output files
* We pull TRAX via `git` with the following and then make the scripts executable:
  ```
  !git clone https://github.com/UCSC-LoweLab/tRAX.git
  !chmod +x -R tRAX
  ```
* The TRAX dependencies can be installed with conda with the following:
  ```
  !mamba env update -n base -f tRAX/trax_env.yaml --quiet && mamba install --quiet -c bioconda gffread pysradb sra-tools
  ```
  * `mamba` is an optional conda package that significantly improves the speed of installing dependencies that can be used in place of `conda`
  * `gffread` also gets installed in this notebook to convert the NCBI gff to a gtf (useful for the demo dataset)
  * `pysradb` is used to quickly and easily pull metadata based on an NCBI accession
  * `sra-tools` is a package used to download fastq files and other experimental data based on accessions

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install() # expect a kernel restart

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!mkdir -p drive/MyDrive/trax_tutorial

In [ ]:
!git clone -b py3test https://github.com/UCSC-LoweLab/tRAX.git
!chmod +x -R tRAX

In [ ]:
!mamba env update -n base -f tRAX/trax_env.yaml --quiet && mamba install --quiet -c bioconda gffread pysradb sra-tools

## Downloading the RNAseq Dataset

In this notebook, we will use RNAseq data from ["Comparative tRNA sequencing and RNA mass spectrometry for surveying tRNA modifications"](https://www.nature.com/articles/s41589-020-0558-1) by Kimura et. al, 2020. This can be accessed via the GEO accession code [GSE147614](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE147614) or its corresponding SRA value SRP254278.

[pysradb](https://github.com/saketkc/pysradb) is a tool to quickly pull SRA metadata for an experiment if given an SRA/ENA/GEO/Etc accession. We will use this to grab the metadata for the experiment and save it as `SRP254278.tsv`.

We only plan to download some of the samples from this experiment. Since the later samples are singleton, we will exclude them and only download the accessions listed below:

```
SRR11431928
SRR11431929
SRR11431930
SRR11431931
SRR11431932
SRR11431933
SRR11431934
SRR11431935
SRR11431936
SRR11431937
```

This list will be saved as `accessions.tsv`

In [ ]:
!pysradb metadata SRP254278 > SRP254278.metadata.tsv
!cat SRP254278.metadata.tsv
!cp SRP254278.metadata.tsv drive/MyDrive/trax_tutorial/.

!grep -v -e "Escherichia coli" -e "trmK" -e "miaA" -e "ttcA" -e "thiI" -e "run_accession" SRP254278.metadata.tsv | cut -f22 > accessions.tsv

[SRA Toolkit](https://github.com/ncbi/sra-tools/wiki/01.-Downloading-SRA-Toolkit) has many valuable tools for working with downloading accession codes. It is best practice to use the `prefetch` command followed by `fastq-dump` or `fasterq-dump` to grab files quickly.

For this tutorial, we will use `-X 100000` to subsample the fastq reads to the first 100,000 total reads so that this pipeline can run quickly. In typical use cases, you should **NOT** use this flag to subset your samples except for testing purposes.

In [ ]:
!mkdir -p fastq
!for i in $(cat accessions.tsv); do prefetch "$i" --output-file fastq/"$i".sra; done
!for i in $(cat accessions.tsv); do fastq-dump -O fastq -X 100000 "$i"; done

# Creating the tRNA Database for TRAX

TRAX requires a tRNA database to run. The files for this can be provided via [gtRNAdb](http://gtrnadb.ucsc.edu/index.html) or generated via [tRNAscan-SE](http://lowelab.ucsc.edu/tRNAscan-SE/). In the case of this tutorial, we will use the [Vibr chol](http://gtrnadb.ucsc.edu/GtRNAdb2/genomes/bacteria/Vibr_chol_O1_biovar_El_Tor_N16961/) database from gtRNAdb.

We will download and extract the tRNA names with `curl` and the reference genome for Vibr Chol. Sometimes, the nomenclature for the reference genome and gtRNAdb don't align to correct for this, we will also use a `sed` command to change the Chromosome names to match gtRNAdb.

Vibro Chol downloaded from NCBI only has a `gff` this needs to be converted into a `gtf` via `gffread`, and then the chromosome names will also need to be converted via `sed` commands.

Finally, we will run `maketrnadb.py` from the tRAX pipeline to create a bowtie2 database for use with tRAX.

In [ ]:
!mkdir -p vibrChol1-tRNAs
!curl -s http://gtrnadb.ucsc.edu/genomes/bacteria/Vibr_chol_O1_biovar_El_Tor_N16961/vibrChol1-tRNAs.tar.gz | tar xzvf - -C vibrChol1-tRNAs/

In [ ]:
!curl -s -OJX GET "https://api.ncbi.nlm.nih.gov/datasets/v2alpha/genome/accession/GCF_000006745.1/download?include_annotation_type=GENOME_FASTA,GENOME_GFF,RNA_FASTA,CDS_FASTA,PROT_FASTA,SEQUENCE_REPORT&filename=GCF_000006745.1.zip" -H "Accept: application/zip"
!unzip -o GCF_000006745.1.zip -d genomes && rm GCF_000006745.1.zip
!sed -i -e "/NC_002505.1/c\>chrI" genomes/ncbi_dataset/data/GCF_000006745.1/GCF_000006745.1_ASM674v1_genomic.fna && sed -i -e "/NC_002506.1/c\>chrII" genomes/ncbi_dataset/data/GCF_000006745.1/GCF_000006745.1_ASM674v1_genomic.fna

In [ ]:
!gffread -E genomes/ncbi_dataset/data/GCF_000006745.1/genomic.gff -T -o genomes/ncbi_dataset/data/GCF_000006745.1/genomic.gtf
!cat genomes/ncbi_dataset/data/GCF_000006745.1/genomic.gtf | sed 's/NC_002505.1/chrI/g' | sed 's/NC_002506.1/chrII/g' | grep -v '^#' > genomes/ncbi_dataset/data/GCF_000006745.1/genes.gtf

In [ ]:
!mkdir -p trnadb
%run tRAX/maketrnadb.py --databasename=trnadb/virboChol1-tRNAdb \
  --genomefile=genomes/ncbi_dataset/data/GCF_000006745.1/GCF_000006745.1_ASM674v1_genomic.fna \
  --trnascanfile=vibrChol1-tRNAs/vibrChol1-tRNAs.out \
  --gtrnafafile=vibrChol1-tRNAs/vibrChol1-tRNAs.fa \
  --namemap=vibrChol1-tRNAs/vibrChol1-tRNAs_name_map.txt \
  --orgmode=bact

The following code will generate a tRAX database. If running locally, the environment you created can be used for future tRAX runs and other Vibro Chol samples. If you plan to analyze more Vibro Chol samples, you can start from the next section of this pipeline. Alternatively, if you wanted to run tRAX on `Hg38`, `hs1`, `mm10`, or another genome, you would need to generate the respective tRNA databases for tRAX first.

# Creating the input files for TRAX

In addition to the fastq files generated from library sequencing, three input files must be created for tRAX to run and act as input configurations.
* The `runfile` is used to specify which fastq files will be provided and their paths so they can be properly trimmed.
* The `samplefile` will determine sample names, sample groups (for normalization), and the path to the trimmed fastq files.
* The `exppairs` determine which groups will be compared in the results output.

## Fastq Runfile

`runfile` These files should be in FASTQ file format and can be compressed by gzip. They are specified in the runfile.

```tsv
fastq/VC_24h_1    fastq/SRR11431928.fastq
fastq/VC_24h_2    fastq/SRR11431929.fastq
fastq/VC_24h_3    fastq/SRR11431930.fastq
fastq/VC_log_1    fastq/SRR11431931.fastq
fastq/VC_log_2    fastq/SRR11431932.fastq
fastq/VC_log_3    fastq/SRR11431933.fastq
fastq/VC_log_4    fastq/SRR11431934.fastq
fastq/VC_CF_1 fastq/SRR11431935.fastq
fastq/VC_CF_2 fastq/SRR11431936.fastq
fastq/VC_CF_3 fastq/SRR11431937.fastq
```

This file is already made and located under the `tutorial` folder in the tRAX GitHub repo called `vibrChol1.run.txt`.

## Samples File

`samplefile` is a three-column tab-delimited file of sample metadata. The columns are:

* Column 1: unique sample replicate name
* Column 2: sample or group name common across replicates
* Column 3: preprocessed FASTQ file name

```tsv
VC_24h_1	VC_24h	fastq/VC_24h_1_trimmed.fastq.gz
VC_24h_2	VC_24h	fastq/VC_24h_2_trimmed.fastq.gz
VC_24h_3	VC_24h	fastq/VC_24h_3_trimmed.fastq.gz
VC_log_1	VC_log	fastq/VC_log_1_trimmed.fastq.gz
VC_log_2	VC_log	fastq/VC_log_2_trimmed.fastq.gz
VC_log_3	VC_log	fastq/VC_log_3_trimmed.fastq.gz
VC_log_4	VC_log	fastq/VC_log_4_trimmed.fastq.gz
VC_CF_1	VC_CF	fastq/VC_CF_1_trimmed.fastq.gz
VC_CF_2	VC_CF	fastq/VC_CF_2_trimmed.fastq.gz
VC_CF_3	VC_CF	fastq/VC_CF_3_trimmed.fastq.gz
```

This file is already made and located under the `tutorial` folder in the tRAX GitHub repo called `vibrChol1.sample.txt`.

## Pairs File

`exppairs` is a two-column tab-delimited file with sample pairs to compare. The columns are:

* Column 1: sample group name
* Column 2: another sample group name

```tsv
VC_24h VC_log
VC_24h VC_CF
VC_log VC_CF
```

The sample group names must match column 2 values in the sample file above. This file is already made and located under the `tutorial` folder in the tRAX GitHub repo called `vibrChol1.pair.txt`.

# Trimming the Fastq Files

Trimming needs to be performed this can be done via the provided `trimadapters.py` function inside of tRAX or by using an adapter trimmer of your choice, such as `cutadapt` or `fastp`.

In [ ]:
%run tRAX/trimadapters.py \
  --runname=vibrChol1_kimura_dedon_waldor \
  --runfile=tRAX/tutorial/vibrChol1.run.txt \
  --firadapter=ACTGTAGGCACCATCAATC \
  --singleend

In [ ]:
!mkdir -p drive/MyDrive/trax_tutorial/trim_ouput/
!cp vibrChol1_kimura_dedon_waldor_* drive/MyDrive/trax_tutorial/trim_ouput/.

# Processing the Samples

The primary function to process the samples and run TRAX is called via `processamples.py`. By default, tRAX uses 8 CPU cores to run `bowtie2` in parallel, this number can be adjusted using the `--cores` flag. The free version of Google Colab gives access to 2 CPU cores; however, this can be adjusted higher based on the resources available on your machine. If more cores are given than the machine has, it will default to the maximum amount available to use.

We call tRAX’s main analysis pipeline with the following code:

In [ ]:
%run tRAX/processsamples.py \
  --experimentname=vibrChol1 \
  --databasename=trnadb/virboChol1-tRNAdb \
  --ensemblgtf=genomes/ncbi_dataset/data/GCF_000006745.1/genes.gtf \
  --samplefile=tRAX/tutorial/vibrChol1.sample.txt \
  --exppairs=tRAX/tutorial/vibrChol1.pair.txt \
  --cores=2

In [ ]:
!cp -R vibrChol1/ drive/MyDrive/trax_tutorial/trax_results/
!cp Rplots.pdf drive/MyDrive/trax_tutorial/.
!cp Rlog-vibrChol1.txt drive/MyDrive/trax_tutorial/.
!cp mismatchcompare.txt drive/MyDrive/trax_tutorial/.
!cp positiondeletions.txt drive/MyDrive/trax_tutorial/.
!cp positionmismatches.txt drive/MyDrive/trax_tutorial/.

# Output results

A full guide to interpreting tRAX results can be found here [http://trna.ucsc.edu/tRAX/outputs/](http://trna.ucsc.edu/tRAX/outputs/). The results of your tRAX run should located in your Google Drive in a folder labeled `trax_tutorial`